#### Import classes

In [1]:
from impl_with_modification import JournalUploadHandler, CategoryUploadHandler, JournalQueryHandler, CategoryQueryHandler, BasicQueryEngine


In [1]:
from impl_with_modification import JournalUploadHandler, CategoryUploadHandler, JournalQueryHandler, CategoryQueryHandler, BasicQueryEngine
path = ' http://10.201.19.160:9999/blazegraph/'
grp_endpoint = f'{path}sparql'
# java -server -Xmx1g -jar blazegraph.jar

# set Journal methods

jou = JournalUploadHandler()
jou.setDbPathOrUrl(grp_endpoint)
jou.pushDataToDb('test_data/doaj.csv')

jou_qh = JournalQueryHandler()
jou_qh.setDbPathOrUrl(grp_endpoint)

# set Category methods

real_path = 'trial.db'
cat = CategoryUploadHandler()
cat.setDbPathOrUrl(real_path)
cat.pushDataToDb('test_data/scimago.json')

cat_qh = CategoryQueryHandler()
cat_qh.setDbPathOrUrl(real_path)

# set Engine methods

que = BasicQueryEngine()
que.addJournalHandler(jou_qh)
que.addCategoryHandler(cat_qh)

True

#### Check JournalQueryHandler

In [2]:
# result_q1 = jou_qh.getJournalsWithTitle('مجلة كلي')
result_q1 = jou_qh.getAllJournals()
# result_q1 = jou_qh.getJournalsWithDOAJSeal()
# result_q1 = jou_qh.getById('2532-2615') # 2532-2615 2385-2615

result_q1

,journal,title,issn,eissn,languages,publisher,license,apc,seal
0,https://comp-data.github.io/resjournal-176,Revista Ciencias Marinas y Costeras,1659-455X,1659-455X,Spanish,"Universidad Nacional, Costa Rica",CC BY-NC-SA,No,No
1,https://comp-data.github.io/resjournal-176,Revista Ciencias Marinas y Costeras,1659-455X,1659-455X,English,"Universidad Nacional, Costa Rica",CC BY-NC-SA,No,No
2,https://comp-data.github.io/resjournal-176,Revista Ciencias Marinas y Costeras,1659-407X,1659-455X,Spanish,"Universidad Nacional, Costa Rica",CC BY-NC-SA,No,No
3,https://comp-data.github.io/resjournal-176,Revista Ciencias Marinas y Costeras,1659-407X,1659-455X,English,"Universidad Nacional, Costa Rica",CC BY-NC-SA,No,No
4,https://comp-data.github.io/resjournal-176,Revista Ciencias Marinas y Costeras,1659-455X,1659-407X,Spanish,"Universidad Nacional, Costa Rica",CC BY-NC-SA,No,No
...,...,...,...,...,...,...,...,...,...
942,https://comp-data.github.io/resjournal-45,Rivista Italiana di Acustica,2385-2615,2385-2615,Italian,Associazione Italiana di Acustica,CC BY,No,No
943,https://comp-data.github.io/resjournal-145,Jurnal Keuangan dan Perbankan,1410-8089,1410-8089,English,Universitas Merdeka Malang,CC BY-SA,No,No
944,https://comp-data.github.io/resjournal-145,Jurnal Keuangan dan Perbankan,2443-2687,1410-8089,English,Universitas Merdeka Malang,CC BY-SA,No,No
945,https://comp-data.github.io/resjournal-145,Jurnal Keuangan dan Perbankan,1410-8089,2443-2687,English,Universitas Merdeka Malang,CC BY-SA,No,No


#### Check CategoryQueryHandler

In [2]:
result_q1 = cat_qh.getAllCat_id_quartile()

# a = "Q1"
result_q1
# print(', '.join(['?'] * len(a)))

,identifiers,category_id
0,1542-4863,Hematology
1,1542-4863,Oncology
2,0007-9235,Hematology
3,0007-9235,Oncology
4,1935-8245,Artificial Intelligence
...,...,...
111584,2307-8960,Medicine (miscellaneous)
111585,1934-1520,Gender Studies
111586,0732-1562,Gender Studies
111587,0215-840X,Law


### Check QueryEngine

In [2]:
# result_q1 = que.getJournalsWithTitle('مجلة كلي')
result_q1 = que.getAllJournals()
# reslt_q1 = que.getJournalsWithDOAJSeal()
# result_q1 = que.getCategory('1542-4863')
result_q1

index number:  0
if...
if...
category list: []
index number:  1
if...
if...
category list: []
index number:  2
if...
if...
category list: [('Analytical Chemistry', 'Q3')]
index number:  3
if...
if...
category list: []
index number:  4
if...
if...
category list: []
index number:  5
if...
if...
category list: []
index number:  6
if...
if...
category list: [('Organic Chemistry', 'Q3')]
index number:  7
if...
if...
category list: []
index number:  8
if...
if...
category list: []
index number:  9
if...
if...
category list: []
index number:  10
if...
if...
category list: []
index number:  11
if...
if...
category list: [('Linguistics and Language', 'Q2'), ('Education', 'Q4')]
index number:  12
if...
if...
category list: []
index number:  13
if...
if...
category list: []
index number:  14
if...
if...
category list: []
index number:  15
if...
if...
category list: []
index number:  16
if...
if...
category list: []
index number:  17
if...
if...
category list: []
index number:  18
if...
if...
cate

KeyboardInterrupt: 

#### edo stuff

In [4]:
import os
import json
import numpy as np
import pandas as pd
from sqlite3 import connect
import SPARQLWrapper
from rdflib import Graph, URIRef, Literal, RDF 
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore

In [8]:
df = pd.read_csv('test_data/doaj.csv')
counter = 0
# df.info()
entrambi_nan_1 = df[['Journal ISSN (print version)', 'Journal EISSN (online version)']].isna().all(axis=1).sum()
# print(entrambi_nan_1)

entrambi_non_nan = ~(df['Journal ISSN (print version)'].isna()) | ~(df['Journal EISSN (online version)'].isna())

# Conta il numero di volte in cui la condizione è True
conteggio = entrambi_non_nan.sum()
# print(conteggio)

seal_count = (df['Journal title'] == 'Revista Ciencias Marinas y Costeras') # Revista Ciencias Marinas y Costeras
df[df['Journal title'] == 'Revista Ciencias Marinas y Costeras']

,Journal title,Journal ISSN (print version),Journal EISSN (online version),Languages in which the journal accepts manuscripts,Publisher,DOAJ Seal,Journal license,APC
176,Revista Ciencias Marinas y Costeras,1659-455X,1659-407X,"English, Spanish","Universidad Nacional, Costa Rica",No,CC BY-NC-SA,No


In [ ]:
    def getCategoriesWithQuartile(self, quartiles: set):
        with connect(self.dbPathOrUrl) as con:
            if not quartiles:
                # Se l'insieme dei quartili in input è vuoto, seleziona tutte le categorie
                query = """
                    SELECT DISTINCT category_internal_id, id, quartile
                    FROM categories
                """
                df = pd.read_sql_query(query, con)
            else:
                # Altrimenti, seleziona solo le categorie con i quartili specificati
                placeholders = ', '.join(['?'] * len(quartiles))
                query = f"""
                    SELECT DISTINCT category_internal_id, id, quartile
                    FROM categories
                    WHERE quartile IN ({placeholders})
                """
                df = pd.read_sql_query(query, con, params=list(quartiles))
        return df

    # Prendere tutte le categorie associate a una lista di aree
    def getCategoriesAssignedToAreas(self, areas: set):
        with connect(self.dbPathOrUrl) as con:
            if not areas:
                query = f"""
                    SELECT DISTINCT c.id, a.area
                    FROM categories c 
                """
                df = pd.read_sql_query(query, con)
            else:
                placeholders = ', '.join(['?'] * len(areas))
                query = f"""
                SELECT DISTINCT c.id, a.area
                FROM categories c
                JOIN areas a ON c.item_internal_id = a.item_internal_id
                WHERE a.area IN ({placeholders})
                """
                df = pd.read_sql_query(query, con, params=list(areas))
        return df

## Cose da vedere con Chiara

- JournalUploadHandler
    - siamo sicuri di dover splittare 'language'?
    - penso si debbano aggiungere delle colonne che traccino hasCategory e hasArea nel db a grafo, mostrale nei test le tabelle che ti sono genute con i metodi e poi vai in BasicMashup a createObjectList (per le parti 0..* creai prima una lista vuota ed if row[hasCategory] != "" crei una nuova lista con i valori interni)

- CI serve una chiave primaria, un id univoco in ogni tabella 

In [31]:
import json
path = 'test_data/scimago.json'
with open(path, 'r') as a:
    file = json.load(a)

counter = list()
for items in file:
    for id in items.get('identifiers'):
        # print(id)
        # break 
        if id == '1474-175X':
            print(items)
    # counter.append(items.get('identifiers'))
# print(len(counter))
    

{'identifiers': ['1474-175X', '1474-1768'], 'categories': [{'id': 'Cancer Research', 'quartile': 'Q1'}, {'id': 'Oncology', 'quartile': 'Q1'}], 'areas': ['Biochemistry, Genetics and Molecular Biology', 'Medicine']}


In [ ]:
import pandas as pd

df = pd.read_csv('test_data/doaj.csv')


,Journal title,Journal ISSN (print version),Journal EISSN (online version),Languages in which the journal accepts manuscripts,Publisher,DOAJ Seal,Journal license,APC
0,Prolíngua,NaN,1983-9979,Portuguese,Universidade Federal da Paraíba,No,CC BY-NC-SA,No
1,Проблеми Законності,2224-9281,2414-990X,"Ukrainian, Russian, English",Yaroslav Mudryi National Law University,No,CC BY,Yes
2,Enlightening Tourism: A Pathmaking Journal,NaN,2174-548X,English,University of Huelva,No,CC BY-NC,No
3,Scientific Journals of the Maritime University...,1733-8670,2392-0378,English,MUS,No,CC BY,Yes
4,"Fronteiras: Journal of Social, Technological a...",NaN,2238-8869,Portuguese,Centro Universitário de Anápolis,No,CC BY-NC,No
...,...,...,...,...,...,...,...,...
193,Revista Analisando em Ciência da Informação,NaN,2317-9708,Portuguese,Universidade Estadual da Paraíba,No,CC BY-NC,No
194,"Rainbow: Journal of Literature, Linguistics an...",2252-6323,2721-4540,English,Universitas Negeri Semarang,No,CC BY,No
195,Jurnal Bidang Pendidikan Dasar,2549-0117,2549-0125,Indonesian,Universitas Kanjuruhan Malang,No,CC BY-SA,Yes
196,Science Diliman,0115-7809,2012-0818,English,University of the Philippines,No,Publisher's own license,No


In [12]:
import json

path = 'test_data/scimago.json'
with open(path, 'r') as f:
    file = json.load(f)

if isinstance(file, dict):
    for key, value in file.items():
        print(f"Chiave: {key}, Valore: {value}")
        break
else:
    print("L'oggetto JSON caricato non è un dizionario.")

L'oggetto JSON caricato non è un dizionario.
